## Import packages

In [1]:
%matplotlib widget

In [2]:
import os
import glob
import numpy  as np
import pandas as pd
import librosa
import librosa.display
import matplotlib.pyplot as plt
import ipywidgets as widgets
import wave
import yaml
import math

### Choose file to inspect

In [3]:
directory='/Users/saroltagabulya/git/Orca/'
wavs=glob.glob('wav_files/*.wav')
file=[]

dropdown=widgets.Dropdown(
    options=wavs,
    description='Wav files:',
    disabled=False,
)

def dropdown_eventhandler(change):
    file.clear()
    file.append(change.new)
    print(change.new)
dropdown.observe(dropdown_eventhandler, names='value')
display(dropdown)

Dropdown(description='Wav files:', options=('wav_files/2015-11-10--15-25.wav', 'wav_files/2017-02-04--10-25-15…

wav_files/2017-02-04--10-25-15--00-05-25--C.wav


### Read metadata


In [4]:
# Set sample rate
try:
    wave_file=wave.open(file[0], "rb")
    sample_rate = wave_file.getframerate()
except:
    sample_rate=int(input('Please check sampling rate manually in the metadata file and set below in Hz! \n'))
    
# Set raw unit of signal
raw_unit=input('Please check the raw unit of the signal and enter below \n ')
    
# Set reference value
ref_value=int(input('Please check reference value in the metadata files and set below!  \n'))

# Calibration value
cal_value=int(input('If available, please check calibration value of hydrophons in the metadata files and set below! \n what does the fullscale voltage correspond to in Pa? \n '))

# Set researcher
researcher= input('Please indicate the researcher who performs the crop by initials [SG, JR] \n ')

Please check sampling rate manually in the metadata file and set below in Hz! 
100000
Please check the raw unit of the signal and enter below 
 V
Please check reference value in the metadata files and set below!  
1
If available, please check calibration value of hydrophons in the metadata files and set below! 
 what does the fullscale voltage correspond to in Pa? 
 1
Please indicate the researcher who performs the crop by initials [SG, JR] 
 SG


### Read in wav file

In [5]:
directory='/Users/saroltagabulya/git/Orca/call_segments/'

folder_name = directory + file[0][10:-4]
os.chdir(folder_name)
wavs=glob.glob('*.wav')
wavs

['2017-02-04--10-28-15--00-00-03--C.wav',
 '2017-02-04--10-28-45--00-00-03--C.wav',
 '2017-02-04--10-29-32--00-00-04--C.wav',
 '2017-02-04--10-30-18--00-00-04--C.wav',
 '2017-02-04--10-28-22--00-00-04--C.wav',
 '2017-02-04--10-28-08--00-00-04--C.wav',
 '2017-02-04--10-28-03--00-00-01--C.wav',
 '2017-02-04--10-29-58--00-00-03--C.wav',
 '2017-02-04--10-28-36--00-00-03--C.wav',
 '2017-02-04--10-29-52--00-00-03--C.wav']

### Choose specific wav

In [6]:
the_wav=wavs[-1]

the_wav

'2017-02-04--10-29-52--00-00-03--C.wav'

### Calculate spectogram

In [7]:
def open_wav(file, sample_rate):
    y, sr = librosa.load(file, sr=sample_rate, mono=False)
    return y, sr

def wav_to_spect_params(y, sr, file, nfft, hop_length, win_length, plot=True): #, call_wav
    
    yml_file=file[:-3] + 'yml'


    with open(yml_file) as f:
        # The FullLoader parameter handles the conversion from YAML
        # scalar values to Python the dictionary format
        metadata = yaml.load(f, Loader=yaml.FullLoader)

    ref=metadata['reference_value']
    raw_unit=metadata['raw_signal_unit']
    start_utc=metadata['start_utc']
    duration_seconds=metadata['duration_seconds']

    
    if len(y)>0:
        # Convert to spectogram 
        
        Y = librosa.stft(y, n_fft=nfft, hop_length=hop_length, win_length=win_length) 
        Ydb = librosa.amplitude_to_db(abs(Y), ref=1)
        

        return Ydb
    
    else: 
        return 'Error with wav'

In [25]:
nffts=[512, 2048, 8192, 2048, 2048, 2048, 2048, 2048, 2048, 2048, 2048, 2048]

hop_l=[32, 32, 32, 1024, 512, 256, 32, 32, 32, 32, 32, 32]

win_l=[512, 2048, 8192,  2048, 2048, 2048, 256, 300, 350, 400, 450, 512]

print(len(nffts), len(hop_l), len(win_l))

12 12 12


### Prepare figure size, read in file

In [44]:
#Prepare plot 
plt.close()
cols=3
rows=math.ceil(len(nffts)/3)
figure, axes = plt.subplots(nrows=rows, ncols=cols, squeeze=False)

print(rows, cols)

#Plot
y, sr=open_wav(the_wav, sample_rate)
for p in range(len(nffts)):
    print(p)
    
    #Define plot position in terms of row and column position
    r=math.floor(p/3)
    c=p%3
    
    print(r, c)
    
    current_axis=axes[r, c]

    Ydb=wav_to_spect_params(y, ref_value, the_wav, nffts[p], hop_l[p], win_l[p], plot=False)

    img = librosa.display.specshow(Ydb, cmap='seismic', sr=sample_rate, x_axis='time', y_axis='fft', ax=current_axis, hop_length=hop_l[p])
    
    if p%3==2: 
        cb=plt.colorbar(img, ax=current_axis) 
        cb.set_label('Power\n[dB re to {} {}^2]'.format(ref_value, raw_unit), rotation=270, labelpad=+15, fontsize=8)
    
    #current_axis.set_xlabel('Time [s]')
    
    #Set axis-limits - specific for where in the plot you want to zoom in!
    y_lim=current_axis.set_ylim()[1]
    x_lim=current_axis.set_xlim()[1]
    current_axis.set_ylim((16000/50000)*y_lim, (20000/50000)*y_lim)
    current_axis.set_xlim((1.450/3)*x_lim, (1.650/3)*x_lim)
    
    # Set y-axis label
    #if p%3==0:
        #current_axis.set_ylabel('Frequency [kHz]', fontsize=9)
    #elif p%3==1 or p%3==2 : 
    
    #current_axis.set_xlabel('Time [s]')
    
    #Set automatic labels to invisible
    current_axis.yaxis.label.set_visible(False)
    current_axis.xaxis.label.set_visible(False)
    
    # Add white legend with parameters
    current_axis.text(0, 0.7, '\n fs: {}kHz\n nfft: {}\n hop_len:{}\n win_len: {}'.format(sample_rate/1000,nffts[p], hop_l[p], win_l[p]), fontsize = 5,  color = 'w', transform=current_axis.transAxes)
    
    current_axis.tick_params(axis='both', which='major', labelsize=5)
    
    
    #convert y-axis ticklabels to kHz!
    locs=current_axis.get_yticks()
    y_ticks = []
    new_yticks=[d/1000 for d in locs]
    current_axis.set_yticklabels(new_yticks)
    
    
    #Set overall labels
    figure.text(0.5, 0.04, 'Time [s]', ha='center')
    figure.text(0.04, 0.5,'Frequency [kHz]', ha='center', rotation=90)
    figure.text(0.96, 0.5,'Power\n[dB re to {} {}^2]'.format(ref_value, raw_unit), ha='center', rotation=270)

   
    
    plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

4 3
0
0 0


<ipython-input-44-3e5484e0a59a>:59: UserWarning: FixedFormatter should only be used together with FixedLocator
  current_axis.set_yticklabels(new_yticks)


1
0 1
2
0 2
3
1 0
4
1 1
5
1 2
6
2 0
7
2 1
8
2 2
9
3 0
10
3 1
11
3 2
